In [1]:
import pandas as pd 
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import plotly.express as px
plt.style.use('ggplot')

from dataloader import load_data, create_count_data

In [2]:
df = load_data()
df.head()

,Station Name,MAC Address,Start Date,End Date,Energy (kWh),Port Type,Port Number,Plug Type,Latitude,Longitude,Fee,Plug In Event Id,Id,Total Duration (min),Charging Time (min),Cluster
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,2011-07-29 20:17:00,2011-07-29 23:20:00,6.249457,Level 2,2,J1772,37.444572,-122.160309,0.0,3,0,183.533333,114.050000,HAMILTON
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,2011-07-30 00:00:00,2011-07-30 00:02:00,0.106588,Level 2,2,J1772,37.444572,-122.160309,0.0,4,1,2.100000,1.900000,HAMILTON
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,2011-07-30 08:16:00,2011-07-30 12:34:00,14.951777,Level 2,2,J1772,37.444572,-122.160309,0.0,5,2,257.533333,257.466667,HAMILTON
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,2011-07-30 14:51:00,2011-07-30 16:55:00,7.159643,Level 2,2,J1772,37.444572,-122.160309,0.0,6,3,123.400000,122.966667,HAMILTON
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,2011-07-30 18:51:00,2011-07-30 20:03:00,1.957765,Level 2,2,J1772,37.444572,-122.160309,0.0,7,4,71.400000,43.900000,HAMILTON
